In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fraudfinder - ML Pipeline

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/vertex_ai/06_formalization.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/vertex_ai/06_formalization.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/vertex_ai/06_formalization.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[Fraudfinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the Fraudfinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

This notebook shows how to use Feature Store, Pipelines and Model Monitoring for building an end-to-end demo using both components defined in `google_cloud_pipeline_components` and custom components. 

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)

Steps performed in this notebook:

* Create a Vetex AI Pipeline to orchestrate and automate the ML workflow

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [1]:
import json

try:
    with open("../config_path.json", "r") as f:
        config_path = json.load(f)
except FileNotFoundError:
    print("config_path.json not found. Please make sure the file exists.")
    ID = None

In [2]:
from utils import read_from_bucket, VertexConfig


config = read_from_bucket(config_path["bucket"], config_path["conf_uri"])
config = VertexConfig(**config)

PROJECT_NUM = !gcloud projects list --filter="$config.PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]

### Delete created Vertex AI resources

Undeploy the model and delete the endpoint

In [ ]:
import google.cloud.aiplatform as vertex_ai
from google.cloud import aiplatform_v1
from google.cloud.aiplatform_v1 import types as aip_types


endpoint = vertex_ai.Endpoint(f"projects/<project-num>/locations/{config.REGION}/endpoints/<endpoint-id>")
endpoint.undeploy("<model-id>")
endpoint.delete()

Deleting Endpoint : projects/401714874268/locations/us-central1/endpoints/6306461146853408768
Endpoint deleted. . Resource name: projects/401714874268/locations/us-central1/endpoints/6306461146853408768
Deleting Endpoint resource: projects/401714874268/locations/us-central1/endpoints/6306461146853408768
Delete Endpoint backing LRO: projects/401714874268/locations/us-central1/operations/953613409797013504
Endpoint resource projects/401714874268/locations/us-central1/endpoints/6306461146853408768 deleted.


Delete the model from registry

In [ ]:
model = vertex_ai.Model(f"projects/<project-num>/locations/{config.REGION}/models/<model-id>")
model.delete()

Deleting Model : projects/401714874268/locations/us-central1/models/5590448179729399808
Model deleted. . Resource name: projects/401714874268/locations/us-central1/models/5590448179729399808
Deleting Model resource: projects/401714874268/locations/us-central1/models/5590448179729399808
Delete Model backing LRO: projects/401714874268/locations/us-central1/models/5590448179729399808/operations/8651391072880033792
Model resource projects/401714874268/locations/us-central1/models/5590448179729399808 deleted.


Remove the experiment

In [ ]:
experiments = vertex_ai.Experiment(experiment_name=config.EXPERIMENT_NAME, project=config.PROJECT_ID, location=config.REGION)
experiments.delete(delete_backing_tensorboard_runs=True)

Deleting Context : projects/401714874268/locations/us-central1/metadataStores/default/contexts/ff-experiment-ucx2v
Context deleted. . Resource name: projects/401714874268/locations/us-central1/metadataStores/default/contexts/ff-experiment-ucx2v
Deleting Context resource: projects/401714874268/locations/us-central1/metadataStores/default/contexts/ff-experiment-ucx2v
Delete Context backing LRO: projects/401714874268/locations/us-central1/metadataStores/default/contexts/ff-experiment-ucx2v/operations/8383426895051489280
Context resource projects/401714874268/locations/us-central1/metadataStores/default/contexts/ff-experiment-ucx2v deleted.


Delete the dataset

In [ ]:
dataset = vertex_ai.TabularDataset(f"projects/<project-num>/locations/{config.REGION}/datasets/<dataset-id>")
dataset.delete()

Deleting TabularDataset : projects/401714874268/locations/us-central1/datasets/5726527587073654784
TabularDataset deleted. . Resource name: projects/401714874268/locations/us-central1/datasets/5726527587073654784
Deleting TabularDataset resource: projects/401714874268/locations/us-central1/datasets/5726527587073654784
Delete TabularDataset backing LRO: projects/401714874268/locations/us-central1/operations/2832740354317352960
TabularDataset resource projects/401714874268/locations/us-central1/datasets/5726527587073654784 deleted.


Delete the feature store and feature groups

In [ ]:
client_options = {"api_endpoint": f"{config.REGION}-aiplatform.googleapis.com"}
client = aiplatform_v1.FeatureOnlineStoreAdminServiceClient(client_options=client_options)
delete_fv_req = aip_types.DeleteFeatureViewRequest(name=f"projects/{config.PROJECT_ID}/locations/{config.REGION}/featureOnlineStores/{config.FEATURESTORE_ID}/featureViews/{config.FEATUREVIEW_ID}")
client.delete_feature_view(request=delete_fv_req)

delete_os_rq = aip_types.DeleteFeatureOnlineStoreRequest(name=f"projects/{config.PROJECT_ID}/locations/{config.REGION}/featureOnlineStores/{config.FEATURESTORE_ID}")
client.delete_feature_online_store(request=delete_os_rq)

In [ ]:
# force delete feature group will delete every mapping it has
client = aiplatform_v1.FeatureRegistryServiceClient(client_options=client_options)
delete_fg_req = aip_types.DeleteFeatureGroupRequest(name=f"projects/{config.PROJECT_ID}/locations/{config.REGION}/featureGroups/{config.TERMINAL_ENTITY_ID}", force=True)
client.delete_feature_group(request=delete_fg_req)
delete_fg_req = aip_types.DeleteFeatureGroupRequest(name=f"projects/{config.PROJECT_ID}/locations/{config.REGION}/featureGroups/{config.CUSTOMER_ENTITY_ID}", force=True)
client.delete_feature_group(request=delete_fg_req)

### Delete BQ datasets

In [33]:
from google.cloud import bigquery

client = bigquery.Client()
client.delete_dataset(dataset=f"{config.PROJECT_ID}.tx", delete_contents=True, not_found_ok=True)
client.delete_dataset(dataset=f"{config.PROJECT_ID}.demographics", delete_contents=True, not_found_ok=True)

### END